In [1]:
import open3d as o3d
import numpy as np
import networkx as nx
from scipy.spatial import cKDTree
from sklearn.neighbors import NearestNeighbors
from collections import defaultdict
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def load_pointcloud(filepath):
    """
    Charge un point cloud depuis un fichier txt
    Format: x y z valeur r g b
    
    Returns:
        points: array (N, 3) des coordonnées xyz
        colors: array (N, 3) des couleurs rgb
        values: array (N,) des valeurs quelconques
    """
    print(f"Chargement du fichier {filepath}...")
    data = np.loadtxt(filepath)
    
    points = data[:, 0:3]  # Colonnes x, y, z
    values = data[:, 3]    # Colonne valeur
    colors = data[:, 4:7]  # Colonnes r, g, b
    
    print(f"✓ {len(points):,} points chargés")
    return points, colors, values

In [3]:
# ============================================================================
# MÉTHODE 1 : KD-TREE / BALL-TREE (Recommandée pour rayon fixe)
# ============================================================================

def build_graph_radius(points, radius=0.1, algorithm='ball_tree'):
    """
    Construit un graphe où deux points sont connectés s'ils sont 
    à une distance < radius
    
    Paramètres:
    -----------
    radius : float
        ⚠️ PARAMÈTRE À AJUSTER ⚠️
        Distance maximale pour créer une arête entre deux points.
        - Trop petit → peu de connexions, graphe fragmenté
        - Trop grand → trop de connexions, calcul lent
        
        💡 Comment choisir:
        1. Calculez la distance moyenne entre voisins proches
        2. Ou utilisez: radius = 2 * moyenne_distance_voisins
        3. Testez avec un échantillon de points d'abord!
        
        Exemple: si vos points sont espacés de ~0.05, essayez radius=0.1
    
    algorithm : 'ball_tree', 'kd_tree', 'brute'
        - 'ball_tree': meilleur pour hautes dimensions et rayon
        - 'kd_tree': rapide pour 3D
        - 'brute': force brute (lent, éviter)
    """
    print(f"\n=== Méthode 1: Radius Neighbors (radius={radius}) ===")
    start = time.time()
    
    nbrs = NearestNeighbors(radius=radius, algorithm=algorithm, n_jobs=-1)
    nbrs.fit(points)
    
    print("Recherche des voisins...")
    distances, indices = nbrs.radius_neighbors(points)
    
    # Construction des arêtes
    edges = []
    edge_weights = []
    
    for i, (neighbors, dists) in enumerate(zip(indices, distances)):
        for j, dist in zip(neighbors, dists):
            if i < j:  # Éviter doublons et self-loops
                edges.append((i, j))
                edge_weights.append(dist)
    
    elapsed = time.time() - start
    print(f"✓ Graphe construit: {len(edges):,} arêtes en {elapsed:.2f}s")
    print(f"  Moyenne arêtes/noeud: {2*len(edges)/len(points):.1f}")
    
    return edges, edge_weights

In [4]:
# ============================================================================
# MÉTHODE 2 : K-PLUS-PROCHES-VOISINS (Alternative)
# ============================================================================

def build_graph_knn(points, k=10, algorithm='kd_tree'):
    """
    Construit un graphe où chaque point est connecté à ses k plus proches voisins
    
    Paramètres:
    -----------
    k : int
        ⚠️ PARAMÈTRE À AJUSTER ⚠️
        Nombre de voisins les plus proches à connecter pour chaque point.
        
        💡 Comment choisir:
        - k=5-10: graphe sparse, peu connecté
        - k=20-50: graphe moyennement connecté
        - k=100+: graphe dense, plus lent
        
        Commencez avec k=10 et ajustez selon vos besoins!
    
    algorithm : 'kd_tree', 'ball_tree', 'brute'
    """
    print(f"\n=== Méthode 2: K-Nearest Neighbors (k={k}) ===")
    start = time.time()
    
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm=algorithm, n_jobs=-1)
    nbrs.fit(points)
    
    print("Recherche des voisins...")
    distances, indices = nbrs.kneighbors(points)
    
    # Construction des arêtes
    edges = set()  # Utilise un set pour éviter doublons
    edge_weights = []
    
    for i, (neighbors, dists) in enumerate(zip(indices, distances)):
        for j, dist in zip(neighbors[1:], dists[1:]):  # Exclut le point lui-même
            edge = tuple(sorted([i, j]))
            if edge not in edges:
                edges.add(edge)
                edge_weights.append(dist)
    
    edges = list(edges)
    elapsed = time.time() - start
    print(f"✓ Graphe construit: {len(edges):,} arêtes en {elapsed:.2f}s")
    print(f"  Moyenne arêtes/noeud: {2*len(edges)/len(points):.1f}")
    
    return edges, edge_weights

In [5]:
# ============================================================================
# MÉTHODE 3 : VOXEL GRID (Pour très gros point clouds)
# ============================================================================

def build_graph_voxel(points, voxel_size=0.1, max_distance=0.15):
    """
    Construit un graphe en utilisant une grille 3D (voxels)
    Plus rapide pour très gros point clouds (>5M points)
    
    Paramètres:
    -----------
    voxel_size : float
        ⚠️ PARAMÈTRE À AJUSTER ⚠️
        Taille des cellules de la grille 3D.
        
        💡 Comment choisir:
        - voxel_size ≈ max_distance: bon compromis
        - Trop petit: trop de voxels, lent
        - Trop grand: trop de points par voxel, lent
        
        Exemple: si max_distance=0.15, essayez voxel_size=0.1
    
    max_distance : float
        ⚠️ PARAMÈTRE À AJUSTER ⚠️
        Distance maximale pour connecter deux points.
        Même principe que 'radius' dans la méthode 1.
    """
    print(f"\n=== Méthode 3: Voxel Grid (voxel_size={voxel_size}, max_dist={max_distance}) ===")
    start = time.time()
    
    # Création du dictionnaire de voxels
    voxel_dict = defaultdict(list)
    for i, point in enumerate(points):
        voxel = tuple((point / voxel_size).astype(int))
        voxel_dict[voxel].append(i)
    
    print(f"  {len(voxel_dict):,} voxels créés")
    print("Recherche des voisins...")
    
    # Construction des arêtes
    edges = set()
    edge_weights = []
    
    for voxel, point_indices in voxel_dict.items():
        # Chercher dans les 27 voxels (3³) autour
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                for dz in [-1, 0, 1]:
                    neighbor_voxel = (voxel[0]+dx, voxel[1]+dy, voxel[2]+dz)
                    
                    if neighbor_voxel in voxel_dict:
                        for i in point_indices:
                            for j in voxel_dict[neighbor_voxel]:
                                if i < j:
                                    dist = np.linalg.norm(points[i] - points[j])
                                    if dist < max_distance:
                                        edges.add((i, j))
                                        edge_weights.append(dist)
    
    edges = list(edges)
    elapsed = time.time() - start
    print(f"✓ Graphe construit: {len(edges):,} arêtes en {elapsed:.2f}s")
    print(f"  Moyenne arêtes/noeud: {2*len(edges)/len(points):.1f}")
    
    return edges, edge_weights

In [6]:
# ============================================================================
# UTILITAIRES
# ============================================================================

def estimate_good_radius(points, sample_size=10000):
    """
    Estime un bon rayon en calculant la distance moyenne au plus proche voisin
    sur un échantillon de points
    """
    if len(points) > sample_size:
        indices = np.random.choice(len(points), sample_size, replace=False)
        sample = points[indices]
    else:
        sample = points
    
    nbrs = NearestNeighbors(n_neighbors=2, algorithm='kd_tree')
    nbrs.fit(sample)
    distances, _ = nbrs.kneighbors(sample)
    
    avg_dist = np.mean(distances[:, 1])  # Distance au 1er voisin (pas self)
    
    print(f"\n📊 Estimation du rayon optimal:")
    print(f"  Distance moyenne au plus proche voisin: {avg_dist:.4f}")
    print(f"  Rayon suggéré (2x): {2*avg_dist:.4f}")
    print(f"  Rayon suggéré (3x): {3*avg_dist:.4f}")
    
    return 2 * avg_dist


def save_graph(edges, edge_weights, output_file):
    """
    Sauvegarde le graphe dans un fichier
    Format: node1 node2 weight
    """
    with open(output_file, 'w') as f:
        for (i, j), w in zip(edges, edge_weights):
            f.write(f"{i} {j} {w:.6f}\n")
    print(f"\n✓ Graphe sauvegardé dans {output_file}")

In [ ]:
input_file = "church.txt"

# Chargement du point cloud
points, colors, values = load_pointcloud(input_file)

# Optionnel: Estimer un bon rayon
suggested_radius = estimate_good_radius(points)

# ========================================================================
# CHOISISSEZ UNE MÉTHODE (décommentez celle que vous voulez)
# ========================================================================

# --- Option 1: Radius Neighbors (Recommandé pour distance fixe) ---
# ⚠️ AJUSTEZ LE PARAMÈTRE 'radius' CI-DESSOUS
edges, weights = build_graph_radius(
    points, 
    radius=0.1,  # 👈 CHANGEZ ICI selon votre échelle
    algorithm='ball_tree'
)

# --- Option 2: K-Nearest Neighbors (Recommandé pour connexions uniformes) ---
# edges, weights = build_graph_knn(
#     points,
#     k=10,  # 👈 CHANGEZ ICI le nombre de voisins
#     algorithm='kd_tree'
# )

# --- Option 3: Voxel Grid (Pour très gros point clouds) ---
# edges, weights = build_graph_voxel(
#     points,
#     voxel_size=0.1,      # 👈 CHANGEZ ICI la taille des voxels
#     max_distance=0.15    # 👈 CHANGEZ ICI la distance max
# )

Chargement du fichier church.txt...
✓ 29,697,591 points chargés

📊 Estimation du rayon optimal:
  Distance moyenne au plus proche voisin: 0.2465
  Rayon suggéré (2x): 0.4930
  Rayon suggéré (3x): 0.7396

=== Méthode 1: Radius Neighbors (radius=0.1) ===


In [ ]:
# ⚠️ CHANGEZ ICI: Chemin de sortie
save_graph(edges, weights, "graph_output.txt")

print("\n✅ Terminé!")

### ChatGPT méthode (trop long ?)

In [ ]:
def load_pointcloud_txt(txt_path):
    """
    Charge un point cloud depuis un fichier txt.

    Format attendu par ligne :
    x y z ? r g b
    """
    data = np.loadtxt(txt_path)

    if data.shape[1] < 7:
        raise ValueError("Le fichier doit contenir au moins 7 colonnes")

    xyz = data[:, 0:3]
    rgb = data[:, 4:7]

    return xyz, rgb


def pointcloud_to_graph_txt(
    txt_path,
    distance_threshold,
    undirected=True
):
    """
    Construit un graphe à partir d'un point cloud txt.

    Args:
        txt_path (str): chemin vers le fichier txt
        distance_threshold (float): seuil de distance
        undirected (bool): graphe non orienté si True

    Returns:
        G (networkx.Graph)
        xyz (np.ndarray)
        rgb (np.ndarray)
    """

    # 1. Charger les données
    xyz, rgb = load_pointcloud_txt(txt_path)
    n_points = xyz.shape[0]

    # 2. KD-Tree pour recherche efficace des voisins
    tree = cKDTree(xyz)

    # 3. Initialisation du graphe
    G = nx.Graph() if undirected else nx.DiGraph()

    # 4. Ajouter les noeuds avec attributs
    for i in range(n_points):
        G.add_node(
            i,
            pos=xyz[i],
            rgb=rgb[i]
        )

    # 5. Ajouter les arêtes selon la distance
    for i in range(n_points):
        neighbors = tree.query_ball_point(xyz[i], distance_threshold)

        for j in neighbors:
            if i == j:
                continue

            if undirected and j < i:
                continue

            dist = np.linalg.norm(xyz[i] - xyz[j])
            G.add_edge(i, j, weight=dist)

    return G, xyz, rgb